### Скрипт преобразования тиковых данных в 5 минутные бары

Vol - объем всего
VB -  объем покупок
VS -  объем продаж
данные с 9:00 14 июня 2022 по 18:50 15 сентября 2022

In [1]:
import pandas as pd
import datetime

In [211]:
df = pd.read_csv('D:/Work/Data/RIU2.csv', delimiter=';', header = 0)
#df = pd.read_csv('D:/Work/Data/RIU2.parquet')

In [212]:
df.head()

,DATE;TIME;LAST;VOL;ID;OPER
0,20220614;100002;116800.000000000;1;19250355535...
1,20220614;100002;116530.000000000;1;19250355535...
2,20220614;100003;116420.000000000;1;19250355535...
3,20220614;100008;117170.000000000;1;19250355535...
4,20220614;100008;117180.000000000;1;19250355535...


In [210]:
df.tail()

,Unnamed: 0,<DATE>;<TIME>;<LAST>;<VOL>;<ID>;<OPER>
6522099,230243,20220915;184914;129240.000000000;2;19250358456...
6522100,230244,20220915;184921;129240.000000000;1;19250358456...
6522101,230245,20220915;184934;129240.000000000;1;19250358456...
6522102,230246,20220915;184939;129240.000000000;7;19250358456...
6522103,230247,20220915;184952;129240.000000000;3;19250358456...


In [173]:
# Удаляем дубликаты
df = df.drop_duplicates()

In [174]:
df.drop(['ID'], axis = 1, inplace = True)

In [175]:
#Добавляем колонки Vol_Buy и Vol_Sell
df.insert(5, "VB", 0)
df.insert(6, "VS", 0)

In [176]:
df.head()

,DATE,TIME,LAST,VOL,OPER,VB,VS
0,20220614,100002,116800.0,1,-1,0,0
1,20220614,100002,116530.0,1,1,0,0
2,20220614,100003,116420.0,1,1,0,0
3,20220614,100008,117170.0,1,-1,0,0
4,20220614,100008,117180.0,1,-1,0,0


In [177]:

df['VB'] = df.apply(lambda row: (row['VOL']*row['OPER'] if row['OPER']==1 # Buy
                                               else 0),
                                  axis=1)

In [178]:

df['VS'] = df.apply(lambda row: (row['VOL']*-row['OPER']
                                               if row['OPER']==-1 # Sell
                                               else 0),
                                  axis=1)

In [179]:
df.head()

,DATE,TIME,LAST,VOL,OPER,VB,VS
0,20220614,100002,116800.0,1,-1,0.0,1.0
1,20220614,100002,116530.0,1,1,1.0,0.0
2,20220614,100003,116420.0,1,1,1.0,0.0
3,20220614,100008,117170.0,1,-1,0.0,1.0
4,20220614,100008,117180.0,1,-1,0.0,1.0


In [180]:
df.tail(10)

,DATE,TIME,LAST,VOL,OPER,VB,VS
6522094,20220915,184740,129250.0,1,1,1.0,0.0
6522095,20220915,184740,129250.0,8,1,8.0,0.0
6522096,20220915,184747,129250.0,1,1,1.0,0.0
6522097,20220915,184803,129240.0,1,1,1.0,0.0
6522098,20220915,184816,129230.0,12,-1,0.0,12.0
6522099,20220915,184914,129240.0,2,1,2.0,0.0
6522100,20220915,184921,129240.0,1,1,1.0,0.0
6522101,20220915,184934,129240.0,1,1,1.0,0.0
6522102,20220915,184939,129240.0,7,1,7.0,0.0
6522103,20220915,184952,129240.0,3,1,3.0,0.0


In [181]:
# Преобразование даты пока она строка в приемлимый формат
df['DATE'] = df['DATE'].map(str)
df['DATE'] = df['DATE'].apply(lambda x: (x[:4]+ '-'+ x[4]+ x[5]+ '-'+ x[6]+ x[7]+ ' '))

In [182]:
# Тоже самое для времени
df['TIME'] = df['TIME'].map(str)
df['TIME'] = df['TIME'].apply(lambda y: (y[0:-4]+ ':'+ y[-4]+ y[-3]+ ':'+ y[-2]+ y[-1]))

In [183]:
df.head()

,DATE,TIME,LAST,VOL,OPER,VB,VS
0,2022-06-14,10:00:02,116800.0,1,-1,0.0,1.0
1,2022-06-14,10:00:02,116530.0,1,1,1.0,0.0
2,2022-06-14,10:00:03,116420.0,1,1,1.0,0.0
3,2022-06-14,10:00:08,117170.0,1,-1,0.0,1.0
4,2022-06-14,10:00:08,117180.0,1,-1,0.0,1.0


In [185]:
# Объединяем дату и время в один столбец
df['Datetime'] = df['DATE'] + df['TIME']

In [186]:
# Преобразование в формат дататайм
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [188]:
df.head()

,DATE,TIME,LAST,VOL,OPER,VB,VS,Datetime
0,2022-06-14,10:00:02,116800.0,1,-1,0.0,1.0,2022-06-14 10:00:02
1,2022-06-14,10:00:02,116530.0,1,1,1.0,0.0,2022-06-14 10:00:02
2,2022-06-14,10:00:03,116420.0,1,1,1.0,0.0,2022-06-14 10:00:03
3,2022-06-14,10:00:08,117170.0,1,-1,0.0,1.0,2022-06-14 10:00:08
4,2022-06-14,10:00:08,117180.0,1,-1,0.0,1.0,2022-06-14 10:00:08


In [189]:
# индекс по Datetime
df = df.assign(Timestamp = pd.to_datetime(df['Datetime'])).set_index('Datetime')

In [190]:
df.tail()

,DATE,TIME,LAST,VOL,OPER,VB,VS,Timestamp
Datetime,,,,,,,,
2022-09-15 18:49:14,2022-09-15,18:49:14,129240.0,2,1,2.0,0.0,2022-09-15 18:49:14
2022-09-15 18:49:21,2022-09-15,18:49:21,129240.0,1,1,1.0,0.0,2022-09-15 18:49:21
2022-09-15 18:49:34,2022-09-15,18:49:34,129240.0,1,1,1.0,0.0,2022-09-15 18:49:34
2022-09-15 18:49:39,2022-09-15,18:49:39,129240.0,7,1,7.0,0.0,2022-09-15 18:49:39
2022-09-15 18:49:52,2022-09-15,18:49:52,129240.0,3,1,3.0,0.0,2022-09-15 18:49:52


In [191]:
# из тиковых данных делаем 5 минутки
min_1 = df.resample('5min')['LAST'].ohlc()

In [192]:
# складываем операции покупки и продажи
min_1v = df.resample('5min')['VOL'].sum() # объем всего, покупки + продажи
min_1vb = df.resample('5min')['VB'].sum() # только покупки
min_1vs = df.resample('5min')['VS'].sum() # только продажи

In [193]:
# все складываем в один df
min_5_out = pd.merge(min_1, min_1v, how='left', on=['Datetime'])

In [194]:
min_5_out = pd.merge(min_5_out, min_1vb, how='left', on=['Datetime'])

In [195]:
min_5_out = pd.merge(min_5_out, min_1vs, how='left', on=['Datetime'])

In [196]:
min_5_out.head()

,open,high,low,close,VOL,VB,VS
Datetime,,,,,,,
2022-06-14 10:00:00,116800.0,117180.0,116000.0,116690.0,317,130.0,187.0
2022-06-14 10:05:00,116410.0,117980.0,116130.0,117770.0,141,100.0,41.0
2022-06-14 10:10:00,117830.0,117980.0,116610.0,116770.0,291,106.0,185.0
2022-06-14 10:15:00,116700.0,116700.0,115160.0,115270.0,261,58.0,203.0
2022-06-14 10:20:00,115210.0,115210.0,114500.0,114850.0,242,99.0,143.0


In [197]:
min_5_out.tail()

,open,high,low,close,VOL,VB,VS
Datetime,,,,,,,
2022-09-15 18:25:00,129230.0,129240.0,129230.0,129240.0,53,29.0,24.0
2022-09-15 18:30:00,129240.0,129240.0,129230.0,129240.0,41,38.0,3.0
2022-09-15 18:35:00,129240.0,129240.0,129230.0,129240.0,37,25.0,12.0
2022-09-15 18:40:00,129240.0,129250.0,129230.0,129250.0,119,113.0,6.0
2022-09-15 18:45:00,129240.0,129250.0,129230.0,129240.0,92,57.0,35.0


In [198]:
min_10_out.sample(10)

,open,high,low,close,VOL,VB,VS
Datetime,,,,,,,
2022-06-27 03:30:00,NaN,NaN,NaN,NaN,0,0.0,0.0
2022-06-20 17:45:00,120440.0,120710.0,120440.0,120630.0,691,520.0,171.0
2022-06-23 08:05:00,NaN,NaN,NaN,NaN,0,0.0,0.0
2022-06-14 16:50:00,113570.0,113710.0,113570.0,113650.0,50,24.0,26.0
2022-07-13 21:15:00,110260.0,110280.0,110250.0,110250.0,11,6.0,5.0
2022-08-29 07:55:00,NaN,NaN,NaN,NaN,0,0.0,0.0
2022-08-20 18:30:00,NaN,NaN,NaN,NaN,0,0.0,0.0
2022-08-27 21:50:00,NaN,NaN,NaN,NaN,0,0.0,0.0
2022-07-25 11:55:00,111720.0,111750.0,111560.0,111750.0,561,283.0,278.0


In [199]:
# Удаляем все строки с NaN
min_5_out = min_10_out.dropna()

In [200]:
min_5_out.sample(10)

,open,high,low,close,VOL,VB,VS
Datetime,,,,,,,
2022-08-29 13:35:00,113750.0,113900.0,113730.0,113900.0,282,209.0,73.0
2022-07-12 17:40:00,111460.0,112240.0,111370.0,112160.0,1469,883.0,586.0
2022-07-01 13:10:00,119530.0,120500.0,119110.0,120460.0,2094,1211.0,883.0
2022-07-15 13:25:00,111750.0,112220.0,111750.0,112180.0,3638,2136.0,1502.0
2022-08-01 11:25:00,103550.0,103810.0,103210.0,103570.0,2826,1388.0,1438.0
2022-07-01 16:25:00,118330.0,118900.0,118150.0,118170.0,1629,666.0,963.0
2022-08-15 13:55:00,104680.0,104920.0,104490.0,104740.0,1456,798.0,658.0
2022-07-22 16:30:00,110030.0,110690.0,109980.0,110640.0,1654,1084.0,570.0
2022-07-05 13:10:00,113280.0,113320.0,112500.0,112940.0,4278,1905.0,2373.0


In [201]:
# Сохраняем в файл
min_10_out.to_csv('D:/Work/Data/RIU2_5min.csv',
                  index= False,
                  sep = ";")